In [1]:
import numpy as np
import string
import nltk
import random
from nltk.stem.porter import PorterStemmer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from datetime import datetime, date
import tensorflow as tf
import json

In [3]:
import json

# Opening JSON file
f = open('/content/intent.json',)

# returns JSON object as
# a dictionary
data = json.load(f)
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hello, thanks for visiting', 'Good to see you again', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'hours', 'patterns': ['What hours are you open?', 'What are your hours?', 'When are you open?'], 'responses': ["We're open every day 9am-9pm", 'Our hours are 9am-9pm every day']}, {'tag': 'name', 'patterns': ['My name is', 'You can call me', 'Everyone calls me'], 'responses': ['Can you please tell me more about you?', 'Describe yourself']}]}


In [4]:
stemmer = PorterStemmer()

In [5]:
stop_words = ['is','am','the','a','an','be','are','were',]


In [6]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [7]:

words = []
y = []
patterns =[]
x = []
classes = []
# getting the classes(tags) output and patterns from 'data'
for intent in data['intents']:
    classes.append(intent['tag'])
    for pattern in intent['patterns']:
# applying tokenizer to convert the sentences into a list of words
        tokens = nltk.word_tokenize(pattern)
# here .extend() is used instead of append() since we don't want to append lists in words
# but its elements i.e words in 'token'
# here 'tokens' is a list of words
        words.extend(tokens)
        patterns.append(pattern)
        y.append(intent['tag'])

# converting to lower case, applyging lemmatization removing the puctuations
# here 'words' is our vocabulary containing all the words
words = [stemmer.stem(word.lower()) for word  in words if word not in string.punctuation and
        word not in stop_words]
# converting the list to set to avoid doubling of words in in 'words'
words = sorted(set(words))
words = list(words)

for list_ in patterns:
    list_ = nltk.word_tokenize(list_)
    list_ = [stemmer.stem(lis.lower()) for lis in list_ if lis not in string.punctuation and lis not in stop_words]
    x.append(list_)


In [8]:
# applying the one hot encoding on our training data
train_x = []
train_y = []
training = []
empty_list = [0]*len(classes)
for idx,list_ in enumerate(x):
    doc = []
    for word in words:
        doc.append(1) if word in list_ else doc.append(0)
    output = list(empty_list)
    output[classes.index(y[idx])] = 1
    training.append([doc,output])
random.shuffle(training)
training = np.array(training,dtype = object)
train_x = list(training[:,0])
train_x = np.array(train_x)
train_y = np.array(list(training[:,1]))

In [9]:
ACCURACY_THRESHOLD = 0.99
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > ACCURACY_THRESHOLD):
            print("\nTraining stopped at epoch number:",epoch)
            self.model.stop_training = True
callbacks = myCallback()


In [10]:
input_shape = (len(train_x[1]),)
output_shape = len(train_y[0])
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
model.fit(x=train_x, y=train_y, epochs=200, verbose=1,callbacks=[callbacks])

Epoch 1/200
1/1 [==============================] - 1s 696ms/step - loss: 1.6771 - accuracy: 0.1765
Epoch 2/200
1/1 [==============================] - 0s 7ms/step - loss: 1.5246 - accuracy: 0.4118
Epoch 3/200
1/1 [==============================] - 0s 8ms/step - loss: 1.3107 - accuracy: 0.6471
Epoch 4/200
1/1 [==============================] - 0s 9ms/step - loss: 1.3532 - accuracy: 0.5294
Epoch 5/200
1/1 [==============================] - 0s 8ms/step - loss: 1.1735 - accuracy: 0.8235
Epoch 6/200
1/1 [==============================] - 0s 12ms/step - loss: 1.0837 - accuracy: 0.6471
Epoch 7/200
1/1 [==============================] - 0s 9ms/step - loss: 0.9786 - accuracy: 0.7059
Epoch 8/200
1/1 [==============================] - 0s 6ms/step - loss: 0.8685 - accuracy: 0.8235
Epoch 9/200
1/1 [==============================] - 0s 9ms/step - loss: 0.7805 - accuracy: 0.9412
Epoch 10/200
1/1 [==============================] - 0s 8ms/step - loss: 0.4892 - accuracy: 0.9412
Epoch 11/200
1/1 [========

In [12]:
import pickle

# Save the Random Forest model to a file
model_filename = "chatbot_model.sav"
pickle.dump(model, open(model_filename,'wb'))

print("Model saved as:", model_filename)

Model saved as: chatbot_model.sav


In [ ]:
# applying bag of words techniques to convert user's text in binary
def bagofwords(msg):
    tokens = nltk.word_tokenize(msg)
    tokens = [stemmer.stem(token.lower()) for token in tokens if token not in string.punctuation and token
              not in stop_words]
    binary_msg = []
    for word in words:
        binary_msg.append(1) if word in tokens else binary_msg.append(0)
    return np.array(binary_msg)


In [ ]:
def prediction(msg):

    message = bagofwords(msg)
    result = model.predict(np.array([message]))
    result = np.argmax(result,axis=1)
    class_index = list(result)[0]
    current_tag = classes[class_index]


    for intent in data['intents']:
        if intent['tag'] == current_tag:
            response = random.choice(intent['responses'])
            return response

In [ ]:
text = input()
response = prediction(text)
while True:
    print(response)
    text = input()
    response = prediction(text)

hi
1/1 [==============================] - 0s 105ms/step
Hello, thanks for visiting
name
1/1 [==============================] - 0s 21ms/step
Describe yourself
goodbye
1/1 [==============================] - 0s 36ms/step
See you later, thanks for visiting
thanks
1/1 [==============================] - 0s 22ms/step
Happy to help!


KeyboardInterrupt: ignored